Here is the code for the constraint solver.

There are several pre-made examples in `cspExamples.py`; below we'll try out one of them.

In [7]:
from aipython.utilities import Displayable
        
class Con_solver(Displayable):
    def __init__(self, csp, domains=None):
        super(Con_solver, self).__init__()
        self.csp=csp
        if domains is not None:
            self.domains = domains
        else:
            self.domains = csp.domains.copy()
        self.max_display_level = 4

    def __repr__(self):
        return str(self.domains)
        
    def make_arc_consistent(self,to_do=None):
        """Makes this CSP arc-consistent using generalized arc consistency
        to_do is a set of (variable,constraint) pairs
        """
        if to_do is None:
            to_do = {(var,const) for const in self.csp.constraints
                             for var in const.scope}
        else:
            to_do = to_do.copy()  # use a copy of to_do
        while to_do:
            var,const = self.choose_arc(to_do)
            self.display(3,"Processing arc (",var,",",const,")")
            other_vars = [ov for ov in const.scope if ov is not var]
            new_domain = {val for val in self.domains[var]
                                  if self.any_holds(const,{var:val},other_vars,0)}
            if new_domain != self.domains[var]:
                self.display(4,"Arc: (",var,",",const,") is inconsistent")
                self.display(3,"Domain pruned","dom(",var,") =",new_domain," due to ",const)
                self.domains[var] = new_domain
                add_to_do = self.new_to_do(var,const)
                to_do |= add_to_do      # set union
                self.display(3,"  adding",add_to_do if add_to_do else "nothing", "to to_do.")
            self.display(4,"Arc: (",var,",",const,") now consistent")
        self.display(2,"AC done. Reduced domains",self.domains)

    def choose_arc(self, to_do):
        #return to_do.pop()
        return self.wait_for_arc_selection(to_do)
    
    def new_to_do(self,var,const):
         """returns new elements to be added to to_do after assigning
         variable var in constraint const.
         """
         return {(nvar,nconst) for nconst in self.csp.var_to_const[var]
                               if nconst != const
                               for nvar in nconst.scope
                               if nvar != var}

    def any_holds(self,const,env,other_vars,ind):
        """returns True if Constraint const holds for an assignment
        that extends env with the variables in other_vars[ind:]
        env is a dictionary
        Warning: this has side effects and changes the elements of env
        """
        if ind==len(other_vars):
            return const.holds(env)
        else:
            var = other_vars[ind]
            for val in self.domains[var]:
                #env = dict_union(env,{var:val})  # no side effects!
                env[var] = val
                if self.any_holds(const,env,other_vars,ind+1):
                    return True
            return False

    def copy_with_assign(self, var=None, new_domain={True,False}):
        """create a copy of the CSP with an assignment var=new_domain
        if var==None then it is just a copy.
        """
        newdoms = self.domains.copy()
        if var:
            newdoms[var] = new_domain
        return Con_solver(self.csp,newdoms)


    def solve_one(self,to_do=None):
        """return a solution to the current CSP or False if there are no solutions
        to_do is the list of arcs to check
        """
        self.make_arc_consistent(to_do)
        if any(len(self.domains[var])==0 for var in self.domains):
            return False
        elif all(len(self.domains[var])==1 for var in self.domains):
            self.display(2,"solution:", {var:select(self.domains[var])  for var in self.domains})
            return {var:select(self.domains[var])  for var in self.domains}
        else:
            var = select(x for x in self.csp.variables if len(self.domains[x])>1)
            if var:
                split = len(self.domains[var])//2
                dom1 = set(list(self.domains[var])[:split]) #a nonempty proper subset
                csp1 = self.copy_with_assign(var,dom1) # copy with dom(var)=dom1
                dom2 = self.domains[var]-dom1
                csp2 = self.copy_with_assign(var,dom2) # copy with dom(var)=dom2
                self.display(3,"...splitting",var,"into",dom1,"and",dom2)
                to_do = self.new_to_do(var,None)
                return csp1.solve_one(to_do) or csp2.solve_one(to_do) 
            

In [8]:
import aispace

from aipython.cspExamples import csp1, csp2, crossword1
csp1.domains['A'] = set([1,3])

csp = aispace.CSPViewer(csp2)
csp.sleep_time = 0.1
csp

In [6]:
import aispace

from aipython.cspExamples import csp1, csp2, crossword1
aispace.CSPViewer(csp2)